In [11]:
import math
import random
import networkx as nx
from networkx import Graph

import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.metrics import mutual_info_score
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from src.pamst import pamst, comp_mst_weight

In [15]:
# Initialize new random graph
n = 100 # Size of the Graph
G: Graph = nx.complete_graph(n)

# Initializing the Edge Weights
for (u, v, w) in G.edges(data=True):
    w['weight'] = random.random()
    

In [40]:
# Compute the Real MST using PRIM
T  = nx.minimum_spanning_tree(G, algorithm ='prim')
weight = 0
for _,_,d in T.edges(data=True):
    weight += d['weight']
print(weight)

#Running PAMST

rho = 1000 # Force Real MST 
sensitivity = 1

noise_lambda = (2 * sensitivity * math.sqrt(n-1))/(math.sqrt(2 * rho))

pamst_edges = pamst(G, sensitivity, noise_lambda) # Gives an iterator which should only be executed once!
pamst_weight = comp_mst_weight(pamst_edges)
print(pamst_weight)

# Post-Processing Sealfon


1.5600286139230728
4.605461345579824
